Analysis Script: Dimension Reduction
Developed by Konstantin Tskhay

------------------------------------------

# Dimension Reduction
## Principal Component Analysis & Exploratory Factor Analysis

### Steps:

1. Import the data
2. Load necessary packages
3. Extract necessary data for PCA
4. Set up all of the preconditions
5. Run PCA/Visualizations
6. Interpret the output
7. Set up for EFA
8. Run EFA/Visualizations
9. Interpret the output
10. Parallel Analysis (Horn, 1965)
11. Mini-validation

-----------------------------

### *Note*. I would like to encourage you type your code--this practice ensures better retention. :)

In [ ]:
#Click in this cell and press Shift + Enter
install.packages("psych")
install.packages("ggplot2")

## Step 1: Import the data

The data are in .csv format: 'new_prof_data.csv'

Here is a brief description of the variables:


1. **ID** = Observation ID
2. **Prof.Name** = The name of the professor. Here, Name1 to Name213 are used
3. **Present** = “Presents the material in an organized, well-planned manner.”
4. **Explain** = “Explains concepts clearly and with appropriate use of examples.”
5. **Communi** = “Communicates enthusiasm and interest in the course material.”
6. **Teach** = “All things considered, performs effectively as a university teacher.”
7. **Workload** = “Compared to other courses at the same level, the workload is…”
8. **Difficulty** = “Compared to other courses at the same level, the level of difficulty of the material is…”
9. **learn.Exp** = “The value of the overall learning experience is…”
10. **Retake** = “Considering your experience with this course, and disregarding your need for it to meet program or degree requirements, would you still have taken this course?”
11. **Inf.** = The aggregate influence score (Interpersonal Charisma Scale)
12. **Kind** = The aggregate kindness score (Interpersonal Charisma Scale)


**_Notes._**

**Q3-Q6 scale**: 1 = extremely poor; 2 = very poor; 3 = poor; 4 = adequate; 5 = good; 6 = very good; 7 = outstanding

**Q7-Q9 scale**: 1 = very low; 2 = low; 3 = below average; 4 = average; 5 = above average; 6 = high; 7 = very high

**Q10 scale**: proportion of people out of 100 who would still take the course considering the experience

**Q11-Q12 scale**: “I am someone who is…”; 1 = strongly disagree; 2 = moderately disagree; 3 = neither agree nor disagree; 4 = moderately agree; 5 = strongly agree



<br>

### Import data

In [ ]:
## Read the data into an object named data
data <- read.csv('https://ibm.box.com/shared/static/t77i51hp8ighrg8mfh2q0gepwcwjfupo.csv') #new_prof_data.csv

## Examine data:
names(data)
str(data)
summary(data)
head(data)


## Step 2: Load necessary packages

This will allow you to use functions needed for factor analysis and principal component analysis

if you don't have these packages on your machine use the following line:

    install.packages('package_name')
    
Where, *package_name* is the name of the package you are interested in

In [ ]:
library(psych) ## for PCA and EFA 
library(ggplot2) ## for some Visuals

## Step 3: Extract necessary data for PCA

This step is simple: select relevant columns.
We want columns 3 through 8.

In [ ]:
names(data) ## look up index
comp.data <- data[,3:8] ## extract data
names(comp.data) ## check

## Step 4: Set up all of the preconditions

**Question**: Can we reduce the number of variables?
**Answer**: Yes. Let's do it.

  

To do so, we need to see interrelationships between the variables.
From before, we know that there may be 2 types of variables emerging.

1. The variables that specify how good the professors are at communication
2. The variables that track the course difficulty

Let us examine whether this may be the case

In [ ]:
round(cor(comp.data), digits = 3) ## produces correlation matrix

You can see immediately that all communication variables are highly correlated. The difficulty variable correlates quite highly with the workload variable. However, there appears to be little overlap between communication and workload/difficulty variables.

**This suggests that there are probably 2 components/factors in our data.**

However, this example is simple because here we see a separation of the variables in the correlation matrix and know that the variables will probably split *a priori*. The data we work with in the real world are more complex, however.

    e.g., 20, 100, 1000, 10000 variables
    
Now, imagine trying to identify how these variables form components or factors using correlation matrix or some type of hypothesis driven reasoning! It is practically impossible!

----------------------------
### Decision Rules:
1. Probably 2 components (Communication, Workload)
2. The components are probably orthogonal
3. Check it empirically

## Step 5 & 6: Run PCA/Visualizations & Interpret the output

In [ ]:
pcaSolution <- prcomp(comp.data, center = TRUE, scale. = TRUE) 

## Produced the object with standard deviatons of all variables
## and Rotation -- or the loadings on the principal components

print(pcaSolution) ## prints PCA solution

pcaSolution$sdev^2

### Let's create the Scree plot: Variance explained versus components

In [ ]:
plot(pcaSolution, type = "l", pch = 16, lwd = 2) ## generates a scree plot

This figure will help us to decide how many components we should extract.
The first two PC explain most of the variability in the data--so, probably 2 components to extract.

Let's see how important each component is in a different way: Take a look of proportion of variance explained (second line in the output table below)

In [ ]:
summary(pcaSolution)


In [ ]:
0.5669 + 0.2829 ## proportion of matrix variance explained by the 
                ## first 2 components

### What items fall on each component?

Well, we can take a look at that examining, pcaSolution's matrix:

In [ ]:
pcaSolution$rotation[,1:2] ## only looks at the first 2 components

**Everything is pretty much as expected!**

**Let's create a visual of the components now:**

In [ ]:
theta <- seq(0,2*pi,length.out = 100)
circle <- data.frame(x = cos(theta), y = sin(theta))
p <- ggplot(circle, aes(x,y)) + geom_path()

loadings <- data.frame(pcaSolution$rotation, .names = row.names(pcaSolution$rotation))
p + geom_text(data=loadings, 
              mapping=aes(x = PC1, y = PC2, label = .names, colour = .names)) +
        coord_fixed(ratio=1) +
        labs(x = "PC1", y = "PC2") +
        theme_bw()

This last step is not necessary, however it is a decent visualization. Basically shows that the 2 factors are orthogonal and that the communication variables tend to map onto the first component and that the difficulty variables tend to map on the second component. Naturally, when you have more variables, the circle will be more complex and less conducive to presentation or interpretation. 

## Step 7: Set up for EFA

Once again, you want to first extract the relevant data. Typically people will do either PCA or EFA, depending on their research question, which informs the selection of the technique. Factor analysis is used when you think that something bigger than the variables at hand causes those variables and interrelationships between those variables. In other words, you assume that your indicators, measured variables, are not measured perfectly. EFA is very common in survey research, including customer satistaction surveys and personality measurement. 

    e.g., suppose your respondents answer some questions about how "outgoing" they are and how "sociable" they are. We assume further that an underlying personality of Extraversion causes people to choose greater number if they are extraverted and lower numbers if they are introveted! That is, people who are extraverted are expected to say that they are more "outgoing" and "sociable" than people who are introverted. We say here, then that extraversion construct (factor) causes systematic variance in reponses to items, but also that the measures are not perfect indicators of extraversion. So what is important here, is the shared variance between the items. 
    
### Factor analysis allows us to identify this latent construct

Okay, now we will:

**1. Extract the relevant data**



In [ ]:
## note that I am using exactly the same code as in PCA, 
## except that I am naming the object differently

names(data) ## look up index
factor.data <- data[,3:8] ## extract data
names(factor.data) ## check

**2. Decide on the number of factors to extract**

In [ ]:
eigen.values <- eigen(cor(factor.data)) ## extract eigenvalues from cor()
eigen.values ## note that eigenvalues are simply squared st.devs from before! 

Create a new scree plot

In [ ]:
plot(eigen.values$values, type="l", ylab = 'Unrotated Eigenvalues',
    xlab = 'Components', lwd = 3)

Once again, we see that 2 factors explain most of the variance. 

Further, you also need to select the factors based on the eigen values themselves they should be greater than 1. Again, the solution is 2. 

In [ ]:
eigen.values$values/(sum(eigen.values$values)) ## compute proportion of 
                                               ##total variance

## How much variance due to the first 2 factors?
sum((eigen.values$values/(sum(eigen.values$values)))[c(1, 2)])

Okay, we meet the three major standards (see parallel analysis later for an alternative result). 

**3. What kind of rotation to use?**

Here, we do not truly need to be exactly data driven. All previous work has demonstrated that orthogonal rotation would be preferable. 

Use: **Varimax** to increase the distance between factors. 

** 4. What would you want as a factoring method?**

Here, we will use principal axis factoring [most commonly used method in psychology research]. 

This says to place factor communalities in the diagonal, which would estimate the residual variation, stating that the factors do not explain all of the variance. In PCA, the 1s are retained across the diagonal. 

### Let's Recap the Pre-processing steps & Decisions:

1. Data extracted
2. Number of factors k = 2
3. Rotation: Orthogonal rotation
4. Factoring method: Principal Axis Factoring (PA)

Let's do it!

## Step 8 & 9: Run EFA/Visualizations & Interpret the output

**Get the correlation matrix**

In [ ]:
corMat <- round(cor(factor.data), digits = 3)
corMat

**Run exploratory factor analysis**

In [ ]:
efaSolution <- fa(r = corMat, nfactors = 2, rotate = "varimax", fm = 'pa')
efaSolution

In [ ]:
pchisq(.39, 4, ncp = 0, lower.tail = FALSE, log.p = FALSE)

The model fit appears to be sufficient. Both factor eigen values are above 1. The loadings on each factor are quite high (>.70 threshold) and the proportion of variance explained == 77%. This is a pretty decent solution. 

Let's do a quick graph for it! 

In [ ]:
fa.diagram(efaSolution)

This is not the best graph and it should not be presented in publication, but it is a good representation of what we did.

Factor 1 causes variables Explain, Teach, Present, and Communi

Factor 2 causes variables Workload, Difficulty

Name your factors. 

## Step 10: Parallel Analysis

This analysis is said to be the most robust analysis for identifying the number of factors. However, it is rarely used. It is rather simple:

1. You will create multiple random datasets with same *n* observations and *k* factors
2. Compute a correlation matrix for each dataset and extract eigenvalues
3. If random eigenvalues are larger than those in the PCA or EFA, you know that the values are likely to be simply noise. 
4. So, you want your eigen values to be larger than those simulated randomly.

Typically 100 simulations is recommended, but more is better. You don't want to overload your machine with computations, however. Here is how to do it.

In [ ]:
fa.parallel(factor.data, n.iter = 100)

A new scree is produced. The red lines represent simulated and resampled data. Blue lines are the actual data. 

As you can see, the line with x-points shows that 2 components should be extracted. -- this line is compared to the top red line and only 2 points exceed the red line--hence, 2 PCs to extract

When it comes to FA, there appears a need for a third component--yet, I am skeptical. The third triangle on the blue line pretty much overlaps with the red line. The distances are much larger in the other cases. 

** Okay, let's run some FA with 3 factors **

In [ ]:
efaSolution2 <- fa(r = corMat, nfactors = 3, rotate = "varimax", fm = 'pa')
efaSolution2

**Output:**

Examinig this output we notice high cross loadings of item Present between PA1 and PA3. Next, the PA3's SS loadings are .81, which does not exceed our threshold = 1. 

Finally, there is little information about the model fit, because it is a saturated model. 

In sum, I typically would not advise for retaining the last factor in the current model. 

## Step 11: Mini-Validation

Here, you want to basically generate factor or component scores with your variables and start building models. I will simply aggregate the variables defined by each component/factor. Then perform linear regression (lm) to see how well my components might predict other variables. 

Create means for Component/factor 1: course communication -- CC
Create means for Component/factor 2: course difficulty -- CD

add them to dataset

In [ ]:
data$CC <- rowMeans(data[, 3:6])
data$CD <- rowMeans(data[, 7:8])

1. Fit the model examining students' learning experience from communication and difficulty
2. Fit the model examining students' retake rate (knowing the course and the professor, would they want to take it again)

In [ ]:
fit1 <- lm(learn.Exp ~ CC + CD, data = data); summary(fit1)
fit2 <- lm(Retake ~ CC + CD, data = data); summary(fit2)

**Output:**

Both are predictors and explain quite a bit of variance;

However, though all students appear to like professors who are better at communication...

They tend to think taht difficulty produced better learning experience, yet would not take the course again. 

Naturally, in more complicated datasets, you can run more complex models. 

<h1, align="center"> FIN </h1>


## RESOURCES:


### Useful Links:

- **Data Science** http://bigdatauniversity.com
- **Clustering** http://bigdatauniversity.com/bdu-wp/bdu-course/machine-learning-cluster-analysis/
- **PCA & CFA** http://bit.ly/1OozM3N

- **R-Code** http://www.statmethods.net/advstats/factor.html
- **Visualize** http://www.r-bloggers.com/computing-and-visualizing-pca-in-r/

### Books:

- **Factor Analysis** http://www.amazon.ca/Factor-Analysis-Richard-L-Gorsuch/dp/1138831999/ref=sr_1_2?ie=UTF8&qid=1444011728&sr=8-2&keywords=factor+analysis
- **Principal Component Analysis** http://www.amazon.ca/Principal-Components-Analysis-George-Dunteman/dp/0803931042/ref=sr_1_2?ie=UTF8&qid=1444011812&sr=8-2&keywords=principal+component+analysis

### Uses in Measurement:

- http://scholarship.sha.cornell.edu/cgi/viewcontent.cgi?article=1618&context=articles
- http://personal.stevens.edu/~ysakamot/719/week4/scaledevelopment.pdf
- http://scholarship.sha.cornell.edu/cgi/viewcontent.cgi?article=1515&context=articles